# AdaBoost and XGBoost models on the loan-level delinquency rates

Explanation
asdf
asdf
a
sdf

asdf

df

asdf

asdf

## Dataset cleaning

Create 4 datasets with 4 different MEI projected values

## Feature Engineering

Sort out categorical variables and other non-numerical

## AdaBoost

## XGBoost

Start with XGBoost model

## Performance Metrics

### Confusion Matrix

### F1-Score